# Этап 1. Загрузка и предобработка данных

In [1]:
from torchvision.datasets import ImageFolder
from torchvision.transforms.v2 import RandomRotation, RandomHorizontalFlip, RandomVerticalFlip, Compose, Resize
from torchvision.transforms import ToTensor, Normalize
from torch.utils.data import DataLoader, Dataset, random_split

In [2]:
train_transforms = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize((0.5), (0.5)),
    RandomHorizontalFlip(p=0.2),
    RandomVerticalFlip(p=0.2),
    RandomRotation([-5, 5], fill=255.)
]) 

val_transforms = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize((0.5), (0.5))
])

class TransformDataset(Dataset):
  def __init__(self, dataset, transforms):
    super(TransformDataset, self).__init__()
    self.dataset = dataset
    self.transforms = transforms

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    x, y = self.dataset[idx]
    return self.transforms(x), y
  
dataset_path = 'ogyeiv2/train'
dataset = ImageFolder(dataset_path)

train_dataset, val_dataset = random_split(dataset, [0.8, 0.2])

train_dataset = TransformDataset(train_dataset, train_transforms)
val_dataset = TransformDataset(val_dataset, val_transforms)

batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False) 

print("Количество изображений в train:", len(train_dataset))
print("Количество изображений в val:", len(val_dataset))
print("Список классов:", dataset.classes)
print("Количество классов:", len(dataset.classes))

Количество изображений в train: 1882
Количество изображений в val: 470
Список классов: ['acc_long_600_mg', 'advil_ultra_forte', 'akineton_2_mg', 'algoflex_forte_dolo_400_mg', 'algoflex_rapid_400_mg', 'algopyrin_500_mg', 'ambroxol_egis_30_mg', 'apranax_550_mg', 'aspirin_ultra_500_mg', 'atoris_20_mg', 'atorvastatin_teva_20_mg', 'betaloc_50_mg', 'bila_git', 'c_vitamin_teva_500_mg', 'calci_kid', 'cataflam_50_mg', 'cataflam_dolo_25_mg', 'cetirizin_10_mg', 'cold_fx', 'coldrex', 'concor_10_mg', 'concor_5_mg', 'condrosulf_800_mg', 'controloc_20_mg', 'covercard_plus_10_mg_2_5_mg_5_mg', 'coverex_4_mg', 'diclopram_75-mg_20-mg', 'dorithricin_mentol', 'dulsevia_60_mg', 'enterol_250_mg', 'favipiravir_meditop_200_mg', 'ibumax_400_mg', 'jutavit_c_vitamin', 'jutavit_cink', 'kalcium_magnezium_cink', 'kalium_r', 'koleszterin_kontroll', 'lactamed', 'lactiv_plus', 'laresin_10_mg', 'letrox_50_mikrogramm', 'lordestin_5_mg', 'merckformin_xr_1000_mg', 'meridian', 'metothyrin_10_mg', 'mezym_forte_10_000_egyseg'

# Этап 2. Объявление модели

In [3]:
from torchsummary import summary
from torchvision.models import resnet101
import torch.nn as nn

model = resnet101(weights='IMAGENET1K_V2')

fc = nn.Linear(in_features=2048, out_features=len(dataset.classes), bias=True)
model.fc = fc

for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

for param in model.layer4.parameters():
    param.requires_grad = True
    
summary(model, input_size=(3, 224, 224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

### Этап 2.1. Переход на GPU

In [5]:
import torch

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Используется устройство: {device}")
model = model.to(device)

Используется устройство: mps


# Этап 3. Дообучение модели

In [6]:
import torch.optim as optim
from tqdm import tqdm

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_one_epoch():
    avg_loss = 0.
    running_loss = 0.

    for i, data in enumerate(tqdm(train_loader)):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    avg_loss = running_loss / (i + 1)
    return avg_loss

EPOCHS = 15

best_vloss = 1e5

for epoch in range(EPOCHS):
    print(f'Эпоха {epoch}')

    model.train()
    avg_loss = train_one_epoch()
    model.eval()
    
    running_vloss = 0.0
    
    with torch.no_grad():
        for i, vdata in enumerate(tqdm(val_loader)):
            vinputs, vlabels = vdata
            vinputs, vlabels = vinputs.to(device), vlabels.to(device)
            voutputs = model(vinputs)
            vloss = criterion(voutputs, vlabels)
            running_vloss += vloss.item()

    avg_vloss = running_vloss / (i + 1)

    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'data/meds_classifier.pt'
        torch.save(model.state_dict(), model_path)
    
    print(f'В конце эпохи ошибка train {avg_loss}, ошибка val {avg_vloss}')

Эпоха 0


100%|██████████| 8/8 [00:23<00:00,  2.93s/it]


В конце эпохи ошибка train 3.9117487192153932, ошибка val 3.058239072561264
Эпоха 1


100%|██████████| 8/8 [00:23<00:00,  2.92s/it]


В конце эпохи ошибка train 2.4168213923772175, ошибка val 1.9506871700286865
Эпоха 2


...

В конце эпохи ошибка train 0.3752324506640434, ошибка val 0.5884230062365532
Эпоха 13


100%|██████████| 8/8 [00:23<00:00,  2.93s/it]


В конце эпохи ошибка train 0.3701956768830617, ошибка val 0.6289954259991646
Эпоха 14


100%|██████████| 8/8 [00:23<00:00,  2.96s/it]


В конце эпохи ошибка train 0.2988275279601415, ошибка val 0.5832013450562954


# Этап 4. Оценка качества

In [8]:
dataset_test_path = '/Users/pavelstepanov/dl_projects/pills_classifier/ogyeiv2/test'
test_dataset = ImageFolder(dataset_test_path)

test_dataset = TransformDataset(test_dataset, val_transforms)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Количество изображений в test:", len(test_dataset))

Количество изображений в test: 504


In [11]:
from sklearn.metrics import classification_report
labels_predicted = []
labels_true = []

model.to('cpu')
model.eval()

with torch.no_grad():
    for data in tqdm(test_loader):
        images, labels = data

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        labels_predicted.extend(predicted.numpy())
        labels_true.extend(labels.numpy())

print(classification_report(labels_true, labels_predicted, target_names=dataset.classes))

100%|██████████| 8/8 [01:20<00:00, 10.09s/it]

                                  precision    recall  f1-score   support

                 acc_long_600_mg       1.00      1.00      1.00         6
               advil_ultra_forte       1.00      1.00      1.00         6
                   akineton_2_mg       0.80      0.67      0.73         6
      algoflex_forte_dolo_400_mg       1.00      0.83      0.91         6
           algoflex_rapid_400_mg       1.00      1.00      1.00         6
                algopyrin_500_mg       1.00      1.00      1.00         6
             ambroxol_egis_30_mg       0.80      0.67      0.73         6
                  apranax_550_mg       1.00      1.00      1.00         6
            aspirin_ultra_500_mg       0.71      0.83      0.77         6
                    atoris_20_mg       0.57      0.67      0.62         6
         atorvastatin_teva_20_mg       0.60      0.50      0.55         6
                   betaloc_50_mg       0.86      1.00      0.92         6
                        bila_git     

### На каких 5 классах модель ошибается чаще всего?
1. teva_ambrobene_30_mg
2. teva_enterobene_2_mg
3. favipiravir_meditop_200_mg
4. narva_sr_1_5_mg_retard
5. jutavit_cink

### Почему модель может ошибаться на этих классах?
Все таблетки на которых большая ошибка, очень похожи друг на друга, без ярко выраженных признаков

### На каких классах модель не совершает ошибок?
На таблетках с ярко выраженными признаками

### Почему эти классы модель распознаёт безошибочно?
Потому что из них легко выделить отличительные признаки

### Как можно улучшить точность классификатора?
Запустить на большее количество эпох, разморозить большее количество слоев для обучения

### Как ещё можно проанализировать результаты и ошибки модели?
Можно вывести фото таблеток и соответствующие предсказанные лейблы для них